In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

In [4]:
from config import ACCESS_KEY,SECRET_KEY,TOKEN,FOLDER,FOLDER_ID,BUCKET_NAME,BUCKET_NAME

In [8]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

headers={'Authorization':TOKEN ,'Accept':'application/json'}

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(month=1)
    return {'key_parquet': yesterday.strftime('year=%Y/month=%m/%d.parquet'),
            'key': yesterday.strftime('year=%Y/month=%m/%d.csv'),
            'key_month': yesterday.strftime('year=%Y/month=%m.csv'),
            'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'last_month_data':last_month_data.strftime('%Y-%m-%d')
            }

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'


def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w', encoding='utf-8')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for row in rows:
            special_str = ','.join("'{0}'".format(i.replace("'", ""))  if isinstance(i, str) else str(i) for i in row)
            temp_file.write(special_str+'\n') 
        offset +=1000 # увеличивает смещение

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=key
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [9]:
s3 = get_s3_instance()

In [10]:
start_date = datetime.datetime.strptime('2024-01-01','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-05-07','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_parquet': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.parquet'),
        'year': pd.date_range(start=start_date, end=end_date).strftime('%Y'),
        'month': pd.date_range(start=start_date, end=end_date).strftime('%m'),
        'day': pd.date_range(start=start_date, end=end_date).strftime('%d'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })

In [12]:
    year = dates_pd.loc[0,['year']].values[0]
    month = dates_pd.loc[0,['month']].values[0]
    day = dates_pd.loc[0,['day']].values[0]

In [13]:
    # Подгрузка companies_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_dir_partner'
    s3_file_name = f"{FOLDER}/{FOLDER}.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка companies_st_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_st_partner'
    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем companies_st_partner и companies_dir_partner

    companies_st_partner_df = companies_st_partner_df.merge(
        companies_dir_partner_df,
        on='partner_uuid',
        how='left'
    )

In [15]:
companies_st_partner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6615 entries, 0 to 6614
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   report_date        6615 non-null   object
 1   partner_uuid       6615 non-null   object
 2   is_blocked         6615 non-null   int64 
 3   pro_subs           6615 non-null   int64 
 4   enterprise_subs    6615 non-null   int64 
 5   billing_pro        6615 non-null   int64 
 6   balance            6615 non-null   int64 
 7   company_name       6587 non-null   object
 8   partner_lk         6615 non-null   int64 
 9   registration_date  6615 non-null   object
 10  tin                654 non-null    object
 11  kpp                376 non-null    object
dtypes: int64(6), object(6)
memory usage: 620.3+ KB


In [ ]:
for i in range(0,dates_pd.shape[0]):
    year = dates_pd.loc[i,['year']].values[0]
    month = dates_pd.loc[i,['month']].values[0]
    day = dates_pd.loc[i,['day']].values[0]
    
    # Подгрузка flats_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'flats_dir_partner'
    s3_file_name = f"{FOLDER}/flats_dir_partner.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    flats_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка flats_st_partner



    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'flats_st_partner'

    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    flats_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем flats_st_partner и flats_dir_partner

    flats_st_partner_df = flats_st_partner_df.merge(
        flats_dir_partner_df[['flat_uuid','address_uuid']],
        on='flat_uuid',
        how='left'
    )


    # Подгрузка entries_installation_points_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'entries_installation_points_dir_partner'
    s3_file_name = f"{FOLDER}/entries_installation_points_dir_partner.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    entries_installation_points_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка installation_point_st_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'installation_point_st_partner'
    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    installation_point_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем entries_installation_points_dir_partner и installation_point_st_partner

    installation_point_st_partner_df = installation_point_st_partner_df.merge(
        entries_installation_points_dir_partner_df[['installation_point_id','address_uuid','city','country','region','parent_uuid']],
        on='installation_point_id',
        how='left'
    )


    # Подгрузка companies_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_dir_partner'
    s3_file_name = f"{FOLDER}/{FOLDER}.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка companies_st_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_st_partner'
    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем companies_st_partner и companies_dir_partner

    companies_st_partner_df = companies_st_partner_df.merge(
        companies_dir_partner_df,
        on='partner_uuid',
        how='left'
    )

    # Объеденяем все данные

    flats_st_partner_df_merged = flats_st_partner_df.merge(
        installation_point_st_partner_df[['digital_keys_count','device_keys_count','monetization','partner_uuid','monetization_is_allowed','installation_point_id','address_uuid','city','country','region','parent_uuid']],
        on='address_uuid',
        how='left'
        )

    columns = ['pro_subs','is_blocked','pro_subs','enterprise_subs','billing_pro','balance','company_name','partner_lk','registration_date','tin','kpp','partner_uuid']
    flats_st_partner_df_merged = flats_st_partner_df_merged.merge(
        companies_st_partner_df[columns],
        on='partner_uuid',
        how='left'
        )

    BUCKET_NAME = "aggregated-data"
    FOLDER = 'flats_research_dashboard'

    local_download_folder= f"{path_on_pc}/{FOLDER}/{dates_pd.loc[i,['date_key_folder']].values[0]}"
    local_download_path = f"{path_on_pc}/{FOLDER}/{dates_pd.loc[i,['date_key_parquet']].values[0]}"

    os.makedirs(local_download_folder, exist_ok=True)

    flats_st_partner_df_merged.to_parquet(local_download_path, compression='snappy', index=False)

    s3_file_path = f"{FOLDER}/{dates_pd.loc[i,['date_key_parquet']].values[0]}"
    pc_file_path = f"{path_on_pc}/{FOLDER}/{dates_pd.loc[i,['date_key_parquet']].values[0]}"

    s3.upload_file(pc_file_path, BUCKET_NAME, s3_file_path)
    print(f'{s3_file_path} the file has already been uploaded!')

In [35]:

    year = dates_pd.loc[0,['year']].values[0]
    month = dates_pd.loc[0,['month']].values[0]
    day = dates_pd.loc[0,['day']].values[0]
    
    # Подгрузка flats_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'flats_dir_partner'
    s3_file_name = f"{FOLDER}/flats_dir_partner.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    flats_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка flats_st_partner



    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'flats_st_partner'

    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    flats_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем flats_st_partner и flats_dir_partner

    flats_st_partner_df = flats_st_partner_df.merge(
        flats_dir_partner_df[['flat_uuid','address_uuid']],
        on='flat_uuid',
        how='left'
    )


    # Подгрузка entries_installation_points_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'entries_installation_points_dir_partner'
    s3_file_name = f"{FOLDER}/entries_installation_points_dir_partner.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    entries_installation_points_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка installation_point_st_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'installation_point_st_partner'
    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    installation_point_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем entries_installation_points_dir_partner и installation_point_st_partner

    installation_point_st_partner_df = installation_point_st_partner_df.merge(
        entries_installation_points_dir_partner_df[['installation_point_id','address_uuid','city','country','region','parent_uuid']],
        on='installation_point_id',
        how='left'
    )


    # Подгрузка companies_dir_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_dir_partner'
    s3_file_name = f"{FOLDER}/{FOLDER}.csv"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:1])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_dir_partner_df = pd.read_csv(f'{local_download_path}')

    # Подгрузка companies_st_partner

    BUCKET_NAME = "dwh-asgard"
    FOLDER = 'companies_st_partner'
    key = f"year={int(year)}/month={int(month)}/{int(day)}.csv"
    s3_file_name = f"{FOLDER}/{key}"
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])

    path_on_pc = 'E:/s3'
    local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc}/{s3_file_name}'

    if os.path.isfile(local_download_path) is False:
    # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        s3.download_file(Bucket=BUCKET_NAME,Key=s3_file_name,Filename=local_download_path)

    companies_st_partner_df = pd.read_csv(f'{local_download_path}')

    # Объеденяем companies_st_partner и companies_dir_partner

    companies_st_partner_df = companies_st_partner_df.merge(
        companies_dir_partner_df,
        on='partner_uuid',
        how='left'
    )

    # Объеденяем все данные

    flats_st_partner_df1 = flats_st_partner_df.merge(
        installation_point_st_partner_df[['digital_keys_count','device_keys_count','monetization','partner_uuid','monetization_is_allowed','installation_point_id','address_uuid','city','country','region','parent_uuid']],
        on='address_uuid',
        how='left'
        )

  
    flats_st_partner_df_merged1 = flats_st_partner_df1.merge(
        companies_st_partner_df[['pro_subs','is_blocked','enterprise_subs','billing_pro','balance','company_name','partner_lk','registration_date','tin','kpp','partner_uuid']],
        on='partner_uuid',
        how='left',
        )

    BUCKET_NAME = "aggregated-data"
    FOLDER = 'flats_research_dashboard'

    local_download_folder= f"{path_on_pc}/{FOLDER}/{dates_pd.loc[0,['date_key_folder']].values[0]}"
    local_download_path = f"{path_on_pc}/{FOLDER}/{dates_pd.loc[0,['date_key_parquet']].values[0]}"

    os.makedirs(local_download_folder, exist_ok=True)

    flats_st_partner_df_merged1.to_parquet(local_download_path, compression='snappy', index=False)

    s3_file_path = f"{FOLDER}/{dates_pd.loc[0,['date_key_parquet']].values[0]}"
    pc_file_path = f"{path_on_pc}/{FOLDER}/{dates_pd.loc[0,['date_key_parquet']].values[0]}"


In [1]:
pip install pyarrow


   ---------------------------------------- 25.9/25.9 MB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
flats_st_partner_df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650004 entries, 0 to 650003
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   report_date              650004 non-null  object 
 1   flat_uuid                650004 non-null  object 
 2   call_blocked             650004 non-null  int64  
 3   blocked                  650004 non-null  int64  
 4   deleted                  650004 non-null  int64  
 5   address_uuid             649995 non-null  object 
 6   digital_keys_count       649945 non-null  float64
 7   device_keys_count        649945 non-null  float64
 8   monetization             649945 non-null  float64
 9   partner_uuid             610769 non-null  object 
 10  monetization_is_allowed  649945 non-null  float64
 11  installation_point_id    649945 non-null  float64
 12  city                     649925 non-null  object 
 13  country                  649925 non-null  object 
 14  regi

In [27]:
flats_st_partner_df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650004 entries, 0 to 650003
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   report_date              650004 non-null  object 
 1   flat_uuid                650004 non-null  object 
 2   call_blocked             650004 non-null  int64  
 3   blocked                  650004 non-null  int64  
 4   deleted                  650004 non-null  int64  
 5   address_uuid             649995 non-null  object 
 6   digital_keys_count       649945 non-null  float64
 7   device_keys_count        649945 non-null  float64
 8   monetization             649945 non-null  float64
 9   partner_uuid             610769 non-null  object 
 10  monetization_is_allowed  649945 non-null  float64
 11  installation_point_id    649945 non-null  float64
 12  city                     649925 non-null  object 
 13  country                  649925 non-null  object 
 14  regi

In [20]:
flats_st_partner_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650004 entries, 0 to 650003
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   report_date              650004 non-null  object 
 1   flat_uuid                650004 non-null  object 
 2   call_blocked             650004 non-null  int64  
 3   blocked                  650004 non-null  int64  
 4   deleted                  650004 non-null  int64  
 5   address_uuid             649995 non-null  object 
 6   digital_keys_count       649945 non-null  float64
 7   device_keys_count        649945 non-null  float64
 8   monetization             649945 non-null  float64
 9   partner_uuid             610769 non-null  object 
 10  monetization_is_allowed  649945 non-null  float64
 11  installation_point_id    649945 non-null  float64
 12  city                     649925 non-null  object 
 13  country                  649925 non-null  object 
 14  regi

In [ ]:
    s3.upload_file(pc_file_path, BUCKET_NAME, s3_file_path)
    print(f'{s3_file_path} the file has already been uploaded!')

In [1]:
python --version

NameError: name 'python' is not defined